In [1]:
import os
import sys
from config import *
import matplotlib
import numpy as np
import pandas as pd
from utils import *
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import KFold

seeds=[200,400,600,800,1000]
train_on_seed=True
train_seed=200
result_dir='problem_classification_results_normalize_False'

results=[]
for dimension in [3,5,10,20]:
    for algorithm in ['DE','ES','PSO']:
        for it in [0,2,4,9,19,29]:
            feature_df=pd.read_csv(f'problem_classification_results_normalize_False/dim_{dimension}_{algorithm}_seeds_200-400-600-800-1000_it_0-{it}')

            for train_seed in seeds:

                global_name=''#f'dim_{dimension}_{argument_algorithm_names}_it_{iteration_min}-{iteration_max}_instance_count_{instance_count}_{"train" if train_on_seed else "test"}_on_seed_{train_seed}'+ ('_differenced' if difference else '')

                kf = KFold(n_splits=10)
                fold=0
                instance_ids=np.array(feature_df.reset_index()['instance_id'].drop_duplicates())
                all_predictions=[]
                all_ys=[]
                accuracies=[]
                importances=[]

                for train_index, test_index in kf.split(instance_ids):

                    run_name=f'{global_name}_fold_{fold}'
                    report_location=f'{result_dir}/{run_name}_report.csv'
                    if os.path.isfile(report_location):
                        fold+=1
                        print(f'Report already exists: {report_location}. Skipping run')
                        continue
                    train,test=get_split_data_for_problem_classification_generalization_testing(instance_ids, train_index, test_index, feature_df, result_dir, run_name, train_seed, train_on_seed)
                    results+=[(dimension,algorithm, it+1, train.shape[0], train.shape[1], test.shape[0], test.shape[1], feature_df.shape[0], feature_df.shape[1])]
                    break
                break

Train/test shape
(2160, 21)
(1200, 21)
Train/test shape
(2160, 53)
(1200, 53)
Train/test shape
(2160, 85)
(1200, 85)
Train/test shape
(2160, 165)
(1200, 165)
Train/test shape
(2160, 325)
(1200, 325)
Train/test shape
(2160, 485)
(1200, 485)
Train/test shape
(2160, 21)
(1200, 21)
Train/test shape
(2160, 53)
(1200, 53)
Train/test shape
(2160, 85)
(1200, 85)
Train/test shape
(2160, 165)
(1200, 165)
Train/test shape
(2160, 325)
(1200, 325)
Train/test shape
(2160, 485)
(1200, 485)
Train/test shape
(2160, 21)
(1200, 21)
Train/test shape
(2160, 53)
(1200, 53)
Train/test shape
(2160, 85)
(1200, 85)
Train/test shape
(2160, 165)
(1200, 165)
Train/test shape
(2160, 325)
(1200, 325)
Train/test shape
(2160, 485)
(1200, 485)
Train/test shape
(2160, 29)
(1200, 29)
Train/test shape
(2160, 77)
(1200, 77)
Train/test shape
(2160, 125)
(1200, 125)
Train/test shape
(2160, 245)
(1200, 245)
Train/test shape
(2160, 485)
(1200, 485)
Train/test shape
(2160, 725)
(1200, 725)
Train/test shape
(2160, 29)
(1200, 29)

In [2]:
results_df=pd.DataFrame(results, columns=['dimension','algorithm','feature_calculation_budget', 'train_instances', 'train_features', 'test_instances', 'test_features','total_instances','total_features'])

In [3]:
results_df.rename(columns={'train_features':'features'})

,dimension,algorithm,feature_calculation_budget,train_instances,features,test_instances,test_features,total_instances,total_features
0,3,DE,1,2160,21,1200,21,12000,21
1,3,DE,3,2160,53,1200,53,12000,53
2,3,DE,5,2160,85,1200,85,12000,85
3,3,DE,10,2160,165,1200,165,12000,165
4,3,DE,20,2160,325,1200,325,12000,325
...,...,...,...,...,...,...,...,...,...
67,20,PSO,3,2160,257,1200,257,12000,257
68,20,PSO,5,2160,425,1200,425,12000,425
69,20,PSO,10,2160,845,1200,845,12000,845
70,20,PSO,20,2160,1685,1200,1685,12000,1685


In [4]:
results_df=results_df.rename(columns={'test_features':'features'})
results_df['features']=results_df['features'].apply(lambda x: x-1)
results_df_features_mean=results_df.groupby(['dimension','feature_calculation_budget']).mean()[['features']]

/tmp/ipykernel_12172/2643194232.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  results_df_features_mean=results_df.groupby(['dimension','feature_calculation_budget']).mean()[['features']]


In [5]:
train_instances=results_df['train_instances'].mean()
test_instances=results_df['test_instances'].mean()

train_instance_df=pd.DataFrame([(train_instances, 'train instances', b) for b in t['feature_calculation_budget'].drop_duplicates().values], columns=['value','variable', 'feature_calculation_budget'])
test_instance_df=pd.DataFrame([(test_instances, 'test instances', b) for b in t['feature_calculation_budget'].drop_duplicates().values], columns=['value','variable', 'feature_calculation_budget'])

NameError: name 't' is not defined

In [ ]:
train_instances

In [ ]:
test_instances

In [ ]:
t=algorithm_results_df_features_mean.reset_index().melt(id_vars=['dimension','feature_calculation_budget'], value_vars=['features'])

In [ ]:
t['variable']=t.apply(lambda x: x['variable'] + ' dim ' + str(x['dimension']), axis=1)
t=pd.concat([t, train_instance_df, test_instance_df])

In [ ]:
plt.figure()
t=t.rename(columns={'value':'count'})
sns.lineplot(t, x='feature_calculation_budget', y='count', hue='variable', style='variable', palette=[light_blue, dark_cyan, lime, dark_blue, '#cc3f50',grey], markers=True, markersize=8)

plt.savefig('feature_instance_count_problem_classification.pdf')

In [ ]:
algorithm_results=[]
for dimension in [3,5,10,20]:
    for it in [0,2,4,9,19,29]:

        for train_seed in seeds:

            global_name=''#f'dim_{dimension}_{argument_algorithm_names}_it_{iteration_min}-{iteration_max}_instance_count_{instance_count}_{"train" if train_on_seed else "test"}_on_seed_{train_seed}'+ ('_differenced' if difference else '')
            result_dir=f'algorithm_classification_results_normalize_False'

            feature_df_file=f'{result_dir}/dim_{dimension}_DE-PSO-ES_seeds_{"-".join([str(s) for s in seeds])}_it_0-{it}'
            feature_df=pd.read_csv(feature_df_file, index_col=[0,1,2,3])
            kf = KFold(n_splits=10)
            df=feature_df.copy()
            fold=0
            problem_ids=np.array(feature_df.reset_index()['problem_id'].drop_duplicates())
            all_predictions=[]
            all_ys=[]
            accuracies=[]
            importances=[]

            for train_index, test_index in kf.split(problem_ids):

                run_name=f'{global_name}_fold_{fold}'

                train,test=get_split_data_for_algorithm_classification_generalization_testing(problem_ids, train_index, test_index, feature_df, result_dir, run_name, train_seed, train_on_seed)


                fold+=1
                algorithm_results+=[(dimension, it+1, train.shape[0], train.shape[1], test.shape[0], test.shape[1], feature_df.shape[0], feature_df.shape[1])]
                break
            break


In [ ]:
algorithm_results_df=pd.DataFrame(algorithm_results, columns=['dimension','feature_calculation_budget', 'train_instances', 'train_features', 'test_instances', 'test_features','total_instances','total_features'])
algorithm_results_df=algorithm_results_df.rename(columns={'test_features':'features'})
algorithm_results_df['features']=algorithm_results_df['features'].apply(lambda x: x-1)
algorithm_results_df_features_mean=algorithm_results_df.groupby(['dimension','feature_calculation_budget']).mean()[['features']]

In [ ]:
t=algorithm_results_df_features_mean.reset_index().melt(id_vars=['dimension','feature_calculation_budget'], value_vars=['features'])

In [ ]:
train_instances=algorithm_results_df['train_instances'].mean()
test_instances=algorithm_results_df['test_instances'].mean()

train_instance_df=pd.DataFrame([(train_instances, 'train instances', b) for b in t['feature_calculation_budget'].drop_duplicates().values], columns=['value','variable', 'feature_calculation_budget'])
test_instance_df=pd.DataFrame([(test_instances, 'test instances', b) for b in t['feature_calculation_budget'].drop_duplicates().values], columns=['value','variable', 'feature_calculation_budget'])

In [ ]:
t['variable']=t.apply(lambda x: x['variable'] + ' dim ' + str(x['dimension']), axis=1)
t=pd.concat([t, train_instance_df, test_instance_df])

In [ ]:
plt.figure()
t=t.rename(columns={'value':'count'})
sns.lineplot(t, x='feature_calculation_budget', y='count', hue='variable', style='variable', palette=[light_blue, dark_cyan, lime, dark_blue, '#cc3f50',grey], markers=True, markersize=8)

plt.savefig('feature_instance_count_algorithm_classification.pdf')